In [3]:
%pip install xgboost lightgbm shap scikit-learn==1.6.1

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 882.6 kB/s eta 0:00:12
   -- ------------------------------------- 0.8/11.1 MB 1.1 MB/s eta 0:00:10
   ---- ----------------------------------- 1.3/11.1 MB 1.3 MB/s eta 0:00:08
   ---- ----------------------------------- 1.3/11.1 MB 1.3 MB/s eta 0:00:08
   ----- ---------------------------------- 1.6/11.1 MB 1.1 MB/s eta 0:00:09
   ----- ---------------------------------- 1.6/11.1 MB 1.1 MB/s eta 0:00:09
   ------ --------------------------------- 1.8/11.1 MB 1.0 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/11.1 MB 1.0 MB/s eta 0:00:09
   -------- ------------------------------- 2.4/11.1 MB 1.0 MB/s eta 0:00:09
   -------- ------------------------------- 2.4/11.1 MB 1.0 MB/s eta 0:00:09
   -------- ------

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
) 
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import HistGradientBoostingClassifier
import joblib
import shap
from scipy.stats import uniform, randint
from datetime import datetime

# Load Dataset with enhanced error handling
print("Loading dataset...")
try:
    data = pd.read_csv("women_night_safety_dataset.csv")
    print("Dataset loaded successfully.")
    print(f"Shape: {data.shape}")
except Exception as e:
    print(f"Error loading dataset: {e}")
    exit()

# Enhanced data exploration
def explore_data(df):
    print("\n=== Data Exploration ===")
    print("\nBasic info:")
    print(df.info())
    print("\nFirst 5 rows:")
    print(df.head())
    print("\nTarget distribution:")
    print(df["Safe_Road"].value_counts(normalize=True))
    print("\nMissing values per column:")
    print(df.isna().sum())
    print("\nDescriptive statistics:")
    print(df.describe())

explore_data(data)

# Feature Engineering
print("\nStarting feature engineering...")
def extract_time_features(df):
    # Convert time to datetime and extract features
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M').dt.time
    df['Hour'] = df['Time'].apply(lambda x: x.hour)
    df['Is_Late_Night'] = df['Hour'].apply(lambda x: 1 if x >= 23 or x <= 4 else 0)
    df['Is_Evening'] = df['Hour'].apply(lambda x: 1 if 18 <= x <= 22 else 0)
    return df

data = extract_time_features(data)

# Preprocessing
print("\nStarting preprocessing...")

# First encode categorical variables before creating interaction terms
categorical_cols = ["Location", "Lighting"]
label_encoders = {}
for col in categorical_cols:
    if col in data.columns:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le

# Now create interaction features with encoded values
data['Lighting_Police'] = data['Lighting'] * data['Police_Presence']
data['Crime_Population'] = data['Crime_Rate'] * data['Population_Density']
print("Feature engineering completed.")

# Handle missing values if any
if data.isna().sum().any():
    print("Handling missing values...")
    # Numerical columns - fill with median
    num_cols = data.select_dtypes(include=['float64', 'int64']).columns
    num_cols = [col for col in num_cols if col not in ["Safe_Road", "Hour", "Is_Late_Night", "Is_Evening"]]
    if len(num_cols) > 0:
        num_imputer = SimpleImputer(strategy='median')
        data[num_cols] = num_imputer.fit_transform(data[num_cols])
    
    # Categorical columns - fill with most frequent
    cat_cols = data.select_dtypes(include=['object']).columns
    if len(cat_cols) > 0:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        data[cat_cols] = cat_imputer.fit_transform(data[cat_cols])

print("Preprocessing completed.")

# Separate features (X) and target (y)
X = data.drop(["Safe_Road", "Time"], axis=1)  # Dropping original Time column
y = data["Safe_Road"]

# Ensure all features are numeric
print("\nFeature types after preprocessing:")
print(X.dtypes)

# Train-Test Split
print("\nSplitting dataset into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
print("Train-Test split completed.")

# Apply SMOTE to balance classes in training set
print("\nApplying SMOTE to balance class distribution...")
try:
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    print("Class distribution after SMOTE:")
    print(pd.Series(y_train_resampled).value_counts())
except Exception as e:
    print(f"Error applying SMOTE: {e}")
    print("Checking for non-numeric values in features...")
    print(X_train.dtypes)
    # Convert any remaining object columns to numeric
    for col in X_train.select_dtypes(include=['object']).columns:
        X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
        X_test[col] = pd.to_numeric(X_test[col], errors='coerce')
    # Try SMOTE again
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    print("Class distribution after SMOTE (after conversion):")
    print(pd.Series(y_train_resampled).value_counts())

# Train model
print("\nTraining model...")
model = HistGradientBoostingClassifier(random_state=42)
model.fit(X_train_resampled, y_train_resampled)

# Save the model
joblib.dump(model, "final_model.pkl")
print("Model saved as final_model.pkl.")

Loading dataset...
Dataset loaded successfully.
Shape: (5000, 10)

=== Data Exploration ===

Basic info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Location                       5000 non-null   object 
 1   Time                           5000 non-null   object 
 2   Lighting                       5000 non-null   object 
 3   Traffic_Density                5000 non-null   float64
 4   Crime_Rate                     5000 non-null   float64
 5   Police_Presence                5000 non-null   int64  
 6   Public_Transport_Availability  5000 non-null   int64  
 7   CCTV_Presence                  5000 non-null   int64  
 8   Population_Density             5000 non-null   float64
 9   Safe_Road                      5000 non-null   int64  
dtypes: float64(3), int64(4), object(3)
memory usage: 390.8+ KB
None

c:\Users\aliar\anaconda3\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Model saved as final_model.pkl.


In [2]:
import sklearn
print(sklearn.__version__)

1.5.1
